### HyDE


HyDE(Hypothetical Document Embeddings) is a retrieval techniqu where, instead of embedding the user's query directly, you first generate a hypothetical answer (document) to the query using an LLM ---

& EMbed the hypothetical document to search your vector store. 

- HyDE - bridges gap b/w user intent & relevant context
- Queries are short
- Languages mismatch b/w query & documents
- You want to retrieve based on answer content, not question words

In [1]:
from langchain_community.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

d:\TrainingPython\Udemy\RAG_Pro\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## 1 . Load & chunk your dataset
chunk_size = 300
chunk_overlap = 100

# loading Data
loader=WikipediaLoader(query="Steve Jobs", load_max_docs=5)
documents=loader.load()

# text splitting
text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
docs=text_splitter.split_documents(documents=documents)
docs

[Document(metadata={'title': 'Steve Jobs', 'summary': 'Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak\'s Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomputers. \nJobs saw the commercial potential 

In [6]:
# 2 . Build Vector STore
from langchain.vectorstores import FAISS
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(docs, embeddings)

In [16]:
# 3. Set up the LLM you'll use to generate hypothetical answers
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
modelUsed = ("openai/gpt-oss-120b")
llm = init_chat_model(
    model=modelUsed,
    model_provider="openai",
    api_key=os.getenv('GROQ_API_KEY'),
    base_url="https://api.groq.com/openai/v1",
    temperature=0.6
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000141E7B14D70>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000141E7B152B0>, root_client=<openai.OpenAI object at 0x00000141E74B2A50>, root_async_client=<openai.AsyncOpenAI object at 0x00000141E7B15010>, model_name='openai/gpt-oss-120b', temperature=0.6, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')

In [17]:
from langchain.vectorstores import Chroma
##creating vector store
db = Chroma.from_documents(documents=docs, 
embedding=embeddings,
persist_directory="output/steve_jobs_for_hyde.db"
)

## Creating a retriever
base_retriever=db.as_retriever(search_kwargs={"k": 5})

In [14]:
from langchain_core.output_parsers import StrOutputParser
## Generating a prompt for generating HyDE
from langchain.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate

def get_hyde_doc(query):
    template=""" Imagine you are an expert writing a detailed explanation on the topic: '{query}'
    create a hypothetical answer for the topic """

    system_message_prompt=SystemMessagePromptTemplate.from_template(template=template)
    chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt])
    messages=chat_prompt.format_prompt(query=query).to_messages()
    print(messages)

    response=llm.invoke(messages)
    hypo_doc=response.content
    return hypo_doc


In [18]:
query="Where was Steve Jobs fired from Apple?"
print(get_hyde_doc(query=query))

[SystemMessage(content=" Imagine you are an expert writing a detailed explanation on the topic: 'Where was Steve Jobs fired from Apple?'\n    create a hypothetical answer for the topic ", additional_kwargs={}, response_metadata={})]
**Short answer:**  
Steve Jobs was effectively fired from Apple at the company’s headquarters in **Cupertino, California** – the Apple campus that served as the corporate headquarters in 1985.

---

## A more detailed look

| Year | Event | Location (Apple’s headquarters at the time) |
|------|-------|---------------------------------------------|
| 1976 | Apple is founded in Jobs’s garage (Los Altos, CA). | – |
| 1977–1982 | Apple moves to a series of rented office spaces as it expands. | Various locations in Cupertino and Los Altos. |
| 1983 | John Sculley is hired as CEO. | Apple’s main office is at **2080 Mariani Avenue, Cupertino, CA** (the “Apple Campus” that pre‑dated the later “Infinite Loop” campus). |
| 1985 (July) | A power struggle between Jobs 

In [19]:
matched_doc=base_retriever.invoke(get_hyde_doc(query))
print(matched_doc)

[SystemMessage(content=" Imagine you are an expert writing a detailed explanation on the topic: 'Where was Steve Jobs fired from Apple?'\n    create a hypothetical answer for the topic ", additional_kwargs={}, response_metadata={})]
[Document(metadata={'title': 'Steve Jobs', 'source': 'https://en.wikipedia.org/wiki/Steve_Jobs', 'summary': 'Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak c

### LangChain-Hypothetical Document Embedder

In [20]:
from langchain.chains.hyde.base import HypotheticalDocumentEmbedder

from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

#Step[1]: Load & Split Documents 
loader=TextLoader("langchain_crewai_dataset.txt")
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
chunks=splitter.split_documents(docs)

In [21]:
# Step[2]: Set up LLm && Embeddings
base_embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\javac\AppData\Local\Temp\ipykernel_7148\3595094574.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  base_embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


According to the official documentation and LangChain source code (mapping in PROMPT_MAP), the default options are:

- web_search
- sci_fact
- arguana
- trec_covid
- fiqa
- dbpedia_entity
- trec_news
- mr_tydi

![image.png](attachment:image.png)

In [22]:
## STep[3] : HyDE Embedder using prompt_key="web_search"
hyde_embedding_func=HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=base_embeddings,
    prompt_key="web_search"
)

In [23]:
#Step[4]: STore Documents in Chroma with HyDE embeddings
vectorstore=Chroma.from_documents(
    documents=chunks,
    embedding=hyde_embedding_func,
    persist_directory="output/langchain"
)

In [24]:
# Step[5]: RAG Answer generation Prompt
rag_prompt=PromptTemplate.from_template(
    """
    Use the context below to answer the question

    Context: {context}

    Question: {input}
    """
)
rag_chain=create_stuff_documents_chain(llm=llm, prompt=rag_prompt)

In [29]:
# STEP[6]: Final RAG Pipeline
def hyde_rag_pipeline(query):
    matched_docs=vectorstore.similarity_search(query, k=4)
    print(matched_docs)

    response=rag_chain.invoke(
        {
            "input": query,
            "context": matched_docs
        }
    )

    return response

In [30]:
# STep[7]: Run example query
query="What memory modules does Langchain provide?"
answer=hyde_rag_pipeline(query)
print("Final Answer:\n", answer)

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v8)'), Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v7)'), Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain offers memory modules like ConversationBufferMemory and ConversationSummaryMemory. These allow the LLM to maintain awareness of previous conversation turns or summarize long interactions to fit within token limits. (v10)'), Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_cont

In [32]:
from langchain.prompts import PromptTemplate
custom = PromptTemplate.from_template(
    "Generate a concise hypothetical answer for this topic: {query}"

)
# STep[3]: HyDE embedder using the "custom prompt"
hyde_embedding_fucntion = HypotheticalDocumentEmbedder.from_llm(
    llm=llm,
    base_embeddings=base_embeddings,
    custom_prompt=custom
)